## Imports and Dataset Download

In [ ]:
!pip install -q pytorch-tabular

In [ ]:
# dataset preprocessing and metric imports
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.utils import shuffle

# xgboost imports
from xgboost import XGBClassifier
from xgboost import plot_importance

# deep learning import
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# basic imports
import matplotlib.pyplot as plt
from typing import Tuple
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
!curl https://opendata.cern.ch/record/328/files/atlas-higgs-challenge-2014-v2.csv.gz -o atlas-higgs-challenge-2014-v2.csv.gz
!gunzip -f atlas-higgs-challenge-2014-v2.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62.5M  100 62.5M    0     0  7161k      0  0:00:08  0:00:08 --:--:-- 9243k


## Data Preparation

In [ ]:
df = pd.read_csv('atlas-higgs-challenge-2014-v2.csv')
feature_columns = [
    'DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h',
    'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet',
    'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau',
    'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt',
    'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi',
    'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
    'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
    'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'
]

label encoding

In [ ]:
df['Label'] = df['Label'].map({'b': 0, 's': 1})
training_data = df[df['KaggleSet'] == 't']
test_data = df[df['KaggleSet'] == 'b']

missing values filled

In [ ]:
training_data = training_data.replace(-999.0, np.nan)
training_data[feature_columns] = training_data[feature_columns].fillna(training_data[feature_columns].median())

feature scaling

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(training_data[feature_columns])
x_scaled = pd.DataFrame(x_scaled, columns=feature_columns)

feature engineering

In [ ]:
pca = PCA(n_components=20, random_state=42)  # Reduce to 20 principal components
x_pca = pca.fit_transform(x_scaled)

balancing the dataset

In [ ]:
# smote = SMOTE(random_state=42)
# X_smote, y_smote = smote.fit_resample(x_pca, training_data['Label'])

In [ ]:
# X, y = shuffle(x_pca, y_smote, random_state=42)
data = pd.DataFrame(x_pca, columns=[f'PC{i+1}' for i in range(x_pca.shape[1])])
data['Label'] = training_data['Label']
data['Weight'] = training_data['Weight']

train, val = train_test_split(data, test_size=0.2, random_state=42)

print("Data Shape:", data.shape)
print("0 Class Instances:", data['Label'].value_counts()[0])
print("1 Class Instances:", data['Label'].value_counts()[1])

print("Training Data Shape:", train.shape)
print("Validation Data Shape:", val.shape)

Data Shape: (250000, 22)
0 Class Instances: 164333
1 Class Instances: 85667
Training Data Shape: (200000, 22)
Validation Data Shape: (50000, 22)


In [ ]:
train_weights = train['Weight']
val_weights = val['Weight']
X_train = train.drop(columns=['Label', 'Weight'], axis=1)
y_train = train['Label']
X_val = val.drop(columns=['Label', 'Weight'], axis=1)
y_val = val['Label']

## Weighted AMS Score

In [ ]:
def weighted_ams_score(y_true, y_pred, weights, b_reg=10):
    """
    Calculate AMS score using event weights

    Parameters:
    -----------
    y_true : array-like
        True labels (0 for background, 1 for signal)
    y_pred : array-like
        Predicted labels (0 for background, 1 for signal)
    weights : array-like
        Event weights for each observation
    b_reg : float
        Regularization term for background
    """
    # Calculate weighted signal (s) and background (b)
    s = np.sum(weights[(y_true == 1) & (y_pred == 1)])  # True positives weighted
    b = np.sum(weights[(y_true == 0) & (y_pred == 1)])  # False positives weighted

    # Calculate AMS
    return np.sqrt(2 * ((s + b + b_reg) * np.log(1 + s/(b + b_reg)) - s))

def evaluate_weighted_predictions(y_true, y_pred, weights):
    """
    Evaluate predictions with weights and print detailed statistics
    """
    # Calculate weighted counts
    weighted_tp = np.sum(weights[(y_true == 1) & (y_pred == 1)])
    weighted_fp = np.sum(weights[(y_true == 0) & (y_pred == 1)])
    weighted_tn = np.sum(weights[(y_true == 0) & (y_pred == 0)])
    weighted_fn = np.sum(weights[(y_true == 1) & (y_pred == 0)])

    print("Weighted Prediction Statistics:")
    print(f"Weighted True Positives: {weighted_tp:.2f}")
    print(f"Weighted False Positives: {weighted_fp:.2f}")
    print(f"Weighted True Negatives: {weighted_tn:.2f}")
    print(f"Weighted False Negatives: {weighted_fn:.2f}")

    # Calculate weighted metrics
    weighted_precision = weighted_tp / (weighted_tp + weighted_fp)
    weighted_recall = weighted_tp / (weighted_tp + weighted_fn)

    print(f"\nWeighted Precision: {weighted_precision:.4f}")
    print(f"Weighted Recall: {weighted_recall:.4f}")

    # Calculate AMS
    ams = weighted_ams_score(y_true, y_pred, weights)
    print(f"\nWeighted AMS Score: {ams:.4f}")

    return ams

## XGBoost Model

In [ ]:
xgb_model = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1,
    gamma=1,
    reg_alpha=0.1,
    reg_lambda=1,
    # scale_pos_weight=99,  # Adjust based on your class imbalance
    random_state=42,
    tree_method='hist',
    eval_metric=['auc', 'aucpr'],
    early_stopping_rounds=20
)

xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=100
)

print('xgb model trained')

[0]	validation_0-auc:0.73879	validation_0-aucpr:0.01136	validation_1-auc:0.74132	validation_1-aucpr:0.01106
[100]	validation_0-auc:0.79724	validation_0-aucpr:0.02032	validation_1-auc:0.79547	validation_1-aucpr:0.01956
[200]	validation_0-auc:0.85332	validation_0-aucpr:0.02536	validation_1-auc:0.85162	validation_1-aucpr:0.02688
[300]	validation_0-auc:0.87481	validation_0-aucpr:0.02808	validation_1-auc:0.87321	validation_1-aucpr:0.02772
[400]	validation_0-auc:0.88464	validation_0-aucpr:0.03026	validation_1-auc:0.88208	validation_1-aucpr:0.02924
[499]	validation_0-auc:0.89518	validation_0-aucpr:0.03265	validation_1-auc:0.89246	validation_1-aucpr:0.03143
xgb model trained


## DNN Model

In [ ]:
class TabularDNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

def train_model(X_train, y_train, X_val, y_val, epochs=50):
    # Get correct input dimension (number of features)
    input_dim = X_train.shape[1]

    # Convert to tensors
    X_train = torch.FloatTensor(X_train)
    y_train = torch.FloatTensor(y_train)
    X_val = torch.FloatTensor(X_val)
    y_val = torch.FloatTensor(y_val)

    # Create dataloaders
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

    # Initialize model and training components
    model = TabularDNN(input_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    criterion = nn.BCELoss()

    best_auc = 0
    for epoch in range(epochs):
        model.train()
        epoch_losses = []

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch).squeeze()
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())

        # Validation
        model.eval()
        with torch.no_grad():
            val_preds = model(X_val).squeeze()
            val_auc = roc_auc_score(y_val, val_preds.numpy())

        scheduler.step(val_auc)

        if val_auc > best_auc:
            best_auc = val_auc
            torch.save(model.state_dict(), 'best_model.pt')

        if epoch % 5 == 0:
            print(f'Epoch {epoch}: Loss = {np.mean(epoch_losses):.4f}, Val AUC = {val_auc:.4f}')

    return model

In [ ]:
model = train_model(X_train.to_numpy(), y_train.to_numpy(), X_val.to_numpy(), y_val.to_numpy())

Epoch 0: Loss = 0.0632, Val AUC = 0.8596
Epoch 5: Loss = 0.0050, Val AUC = 0.9201
Epoch 10: Loss = 0.0049, Val AUC = 0.9277
Epoch 15: Loss = 0.0048, Val AUC = 0.9302
Epoch 20: Loss = 0.0048, Val AUC = 0.9312
Epoch 25: Loss = 0.0047, Val AUC = 0.9323
Epoch 30: Loss = 0.0047, Val AUC = 0.9337
Epoch 35: Loss = 0.0047, Val AUC = 0.9342
Epoch 40: Loss = 0.0046, Val AUC = 0.9341
Epoch 45: Loss = 0.0046, Val AUC = 0.9345


## Ensemble of DNN and XGBoost

In [ ]:
def preprocess_test_data(X_test, scaler, pca):
    X_test = X_test.replace(-999.0, np.nan)
    X_test[feature_columns] = X_test[feature_columns].fillna(X_test[feature_columns].median())
    X_scaled = scaler.transform(X_test)
    X_processed = pca.transform(X_scaled)
    return X_processed

In [ ]:
def ensemble_predict(xgb_model, dnn_model, test_df, scaler, feature_cols, pca=None, threshold=0.5, weights=[0.5, 0.5]):
    X_processed = preprocess_test_data(test_df, scaler, feature_cols, pca)
    xgb_probs = xgb_model.predict_proba(X_processed)[:, 1]

    X_torch = torch.FloatTensor(X_processed)
    dnn_model.eval()
    with torch.no_grad():
        dnn_probs = dnn_model(X_torch).squeeze().numpy()

    # Combine
    ensemble_probs = weights[0] * xgb_probs + weights[1] * dnn_probs
    ensemble_preds = (ensemble_probs > threshold).astype(int)

    return ensemble_preds, ensemble_probs

In [ ]:
X_test = test_data[feature_columns]
y_test = test_data['Label']
weights = test_data['Weight']

ensemble_preds, probs = ensemble_predict(
    xgb_model, model, X_test,
    scaler=scaler,
    pca=pca,
    weights=[0.5, 0.5]
)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [ ]:
evaluate_weighted_predictions(y_test, ensemble_preds, weights)
print(f"\n\nPrecision: {precision_score(y_test, ensemble_preds)}, Recall: {recall_score(y_test, ensemble_preds)}")

Weighted Prediction Statistics:
Weighted True Positives: 0.00
Weighted False Positives: 0.00
Weighted True Negatives: 50463.91
Weighted False Negatives: 84.25

Weighted Precision: nan
Weighted Recall: 0.0000

Weighted AMS Score: 0.0000


Precision: 0.0, Recall: 0.0


<ipython-input-171-900da544d6c5>:40: RuntimeWarning: invalid value encountered in scalar divide
  weighted_precision = weighted_tp / (weighted_tp + weighted_fp)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
ensemble_preds

array([0, 0, 0, ..., 0, 0, 0])